# Descriptors

Descriptors specify what kinds of values are valid for a parameter.
There are currently, three variants: `RangeParameter`, `DependentParameter` and `CategoryParameter`.

In [ ]:

from besos import sampling
from besos.evaluator import EvaluatorGeneric
from besos.parameters import (
    CategoryParameter,
    DependentParameter,
    Parameter,
    RangeParameter,
)
from besos.problem import Problem

### RangeParameters
$min \leq x \leq max$

In [ ]:
zero_to_one_exclusive = RangeParameter(min_val=0.01, max_val=0.99)

### DependentParameter

The value of dependent parameter depends on a range parameter.

For mode = 'sum', x = factor - target_val

For mode = 'subtract', x = target_val - factor

For mode = 'multiple', x = factor * target_val

For mode = 'power', x = target_val ^ factor

index represents the index of the target range parameter in the parameter list

Check out [this](Evaluators/DependParamAndNonObj.ipynb) notebook for more examples.

In [ ]:
dp = DependentParameter(mode="sum", factor=1, index=0)

### CategoryParameters
A list of options.

In [ ]:
text_example = CategoryParameter(options=["a", "b", "c", "other"])
single_digit_integers = CategoryParameter(options=range(10))

### Sampling
These descriptors can be used to make `Parameters`.
Then we can generate samples.

In [ ]:
parameters = [
    Parameter(value_descriptor=zero_to_one_exclusive, name="0-1"),
    Parameter(value_descriptor=dp, name="dp"),
    Parameter(value_descriptor=single_digit_integers, name="single digit"),
    Parameter(value_descriptor=text_example, name="text"),
]
problem = Problem(parameters, outputs=["output"])

samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=10)
samples

### Evaluation
Since we did not specify selectors for the parameters, we cannot evaluate them using an EnergyPlus simulation.
Instead, we will use a custom evaluation function.

In [ ]:
def evaluation_function(values):
    v, x, y, z = values
    if z == "other":
        return (v,), ()
    else:
        return (x * y,), ()


evaluator = EvaluatorGeneric(evaluation_function, problem)
# The evaluator will use this objective by default
outputs = evaluator.df_apply(samples, keep_input=True)
# outputs is a pandas dataframe with one column since only one objective was requested

In [ ]:
outputs